## 1. Imports

In [16]:
import os
import time
import json
import requests
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool

## 2. Define helper function

In [ ]:
# Function for the agent to call
def fetch_weather(location: str) -> str:
    """Fetches the weather information for the specified location using WeatherAPI.com.
    
    :param location: The location to fetch weather for.
    :return: Weather information as a JSON string.
    """
    api_key = "your-api-key"
    base_url = "http://api.weatherapi.com/v1/current.json"
    
    if not api_key:
        return json.dumps({
            "error": "Weather API key not found. Please set WEATHER_API_KEY environment variable.",
            "location": location
        })
    
    try:
        params = {
            'key': api_key,
            'q': location,
            'aqi': 'no'
        }
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        temperature = data['current']['temp_c']
        weather_condition = data['current']['condition']['text']
        humidity = data['current']['humidity']
        wind_speed = data['current']['wind_kph']
        
        weather_info = f"{weather_condition}, {temperature}°C, Humidity: {humidity}%, Wind: {wind_speed} km/h"
        
        return json.dumps({
            "weather": weather_info,
            "location": location,
            "temperature": temperature,
            "condition": weather_condition,
            "humidity": humidity,
            "wind_speed": wind_speed
        })
        
    except requests.exceptions.RequestException as e:
        return json.dumps({
            "error": f"Failed to fetch weather data: {str(e)}",
            "location": location
        })
    except KeyError as e:
        return json.dumps({
            "error": f"Unexpected API response format: {str(e)}",
            "location": location
        })
    except Exception as e:
        return json.dumps({
            "error": f"Error fetching weather data: {str(e)}",
            "location": location
        })


## 3. Initialize project client


In [19]:
# Load configuration (using the same pattern as your BMI example)
def find_cred_json(start_path):
    current = Path(start_path)
    while current != current.parent:
        cred_file = current / 'cred.json'
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

In [20]:
# Initialize project client
try:
    start_dir = os.getcwd()
    file_path = find_cred_json(start_dir)
    
    if not file_path:
        raise FileNotFoundError("Could not find cred.json file")
    
    print(f"Found cred.json at: {file_path}")
    
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)
    
    project_client = AIProjectClient(
        credential=DefaultAzureCredential(),
        endpoint=loaded_config["PROJECT_ENDPOINT"],
    )
    
    model_name = loaded_config.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")
    print("✅ Successfully initialized AIProjectClient")
    
except Exception as e:
    print(f"❌ Error initializing project client: {e}")
    exit(1)

# Define user functions
user_functions = {fetch_weather}

# Initialize the FunctionTool with user-defined functions
functions = FunctionTool(functions=user_functions)

Found cred.json at: /workspaces/Azure-ai-foundry01/cred.json
✅ Successfully initialized AIProjectClient


## 4.Create agent

In [21]:
def create_weather_agent():
    """Create an agent with weather function capabilities."""
    try:
        # Create an agent with custom functions
        agent = project_client.agents.create_agent(
            model=model_name,
            name="weather-agent",
            instructions="You are a helpful weather assistant. Use the fetch_weather function to get weather information for any location the user asks about.",
            tools=functions.definitions,
        )
        print(f"✅ Created agent, ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        return None

## 5.Create thread

In [22]:
def run_weather_conversation(agent, user_message):
    """Run a conversation with the weather agent."""
    try:
        # Create a thread for communication
        thread = project_client.agents.threads.create()
        print(f"📝 Created thread, ID: {thread.id}")

        # Send a message to the thread
        message = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_message,
        )
        print(f"➕ Created message, ID: {message.id}")

        # Create and process a run for the agent to handle the message
        run = project_client.agents.runs.create(
            thread_id=thread.id, 
            agent_id=agent.id
        )
        print(f"▶️ Created run, ID: {run.id}")

        # Poll the run status until it is completed or requires action
        max_attempts = 60
        attempts = 0
        
        while run.status in ["queued", "in_progress", "requires_action"] and attempts < max_attempts:
            time.sleep(1)
            attempts += 1
            run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
            
            if run.status == "requires_action":
                print("🔧 Processing function calls...")
                tool_calls = run.required_action.submit_tool_outputs.tool_calls
                tool_outputs = []
                
                for tool_call in tool_calls:
                    if tool_call.function.name == "fetch_weather":
                        # Parse the arguments
                        args = json.loads(tool_call.function.arguments)
                        location = args.get("location", "New York")
                        print(f"   Fetching weather for: {location}")
                        
                        # Call the function
                        output = fetch_weather(location)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id, 
                            "output": output
                        })
                
                # Submit the tool outputs
                project_client.agents.runs.submit_tool_outputs(
                    thread_id=thread.id, 
                    run_id=run.id, 
                    tool_outputs=tool_outputs
                )
                print("   ✅ Submitted tool outputs")

        print(f"🤖 Run completed with status: {run.status}")

        # Fetch and display all messages from the thread
        if run.status == "completed":
            messages = project_client.agents.messages.list(thread_id=thread.id)
            message_list = list(messages)
            
            print("\n💬 Conversation:")
            for message in message_list:
                role = message.role
                if message.content:
                    for content in message.content:
                        if hasattr(content, 'text') and content.text:
                            print(f"   {role.upper()}: {content.text.value}")
        
        return thread, run
        
    except Exception as e:
        print(f"❌ Error in conversation: {e}")
        import traceback
        traceback.print_exc()
        return None, None

## 6.Run agent and handle tool calls

In [24]:
# Main execution
if __name__ == "__main__":
    # Create the weather agent
    weather_agent = create_weather_agent()
    
    if weather_agent:
        # Test the agent with a weather query
        user_query = "What's the weather like in New York? Also tell me about London."
        print(f"\n🗣️ User query: {user_query}")
        
        thread, run = run_weather_conversation(weather_agent, user_query)
        
        # Cleanup
        try:
            project_client.agents.delete_agent(weather_agent.id)
            print("\n🗑️ Deleted weather agent")
        except Exception as e:
            print(f"⚠️ Could not delete agent: {e}")

✅ Created agent, ID: asst_rUxbzPLDc9IfSTRduRBx1Cq0

🗣️ User query: What's the weather like in New York? Also tell me about London.
📝 Created thread, ID: thread_CulfVfVrjkEo4B3SDv6Ubi9Z
➕ Created message, ID: msg_iTjl4GhrEdEEEre2Z0iYufLg
▶️ Created run, ID: run_dRCc4pkjzdw1IORTIm9PTeZo
🔧 Processing function calls...
   Fetching weather for: New York
   Fetching weather for: London
   ✅ Submitted tool outputs
🤖 Run completed with status: RunStatus.COMPLETED

💬 Conversation:
   ASSISTANT: Here is the current weather update:

- **New York**: It's partly cloudy with a temperature of 22.2°C. The humidity is at 64%, and the wind is blowing at 14.0 km/h.

- **London**: It's partly cloudy as well, with a slightly cooler temperature of 19.1°C. The humidity stands at 56%, and there's a moderate wind speed of 24.8 km/h.
   USER: What's the weather like in New York? Also tell me about London.

🗑️ Deleted weather agent
